# Connect Four


## Monte Carlo Tree Search

In [1]:
from collections import defaultdict
import random
import math
import functools 

### Class Board

In [16]:
class Board(defaultdict):
    empty = '.'
    used = '#'
    
    def __init__(self, width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
 
    def __missing__(self, pos):
        (x,y) = pos
        if (x>=0) and (y>=0) and (x<self.width) and (y<self.height):
            return self.empty
        else:
            return self.used  
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.current_player)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'
        
    def update_board(self, changes: dict, **kwds) -> 'Board':
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

In [17]:
def k_in_row(board, player, square, k):
    
    def in_row(x,y,xpos,ypos):
        if board[x,y] != player:
            return 0
        else:
            return 1 + in_row(x + xpos, y + ypos,xpos,ypos)
    
    
    lista = [] 
    for xpos, ypos in (1, 1),(0, 1),(1, 0),(1, -1):  
        lista.append(in_row(*square, xpos, ypos) + in_row(*square, -xpos, -ypos) - 1 >= k) 
    for i in lista:
        if i:
            return True
    return False 

In [18]:
class TicTacToe():
    def __init__(self, height=3, width=3, k=3):
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)
 
    def to_move(self, board):
        return self.squares - set(board)
    
    def utility(self, board, player):
        if player == 'X':
            return board.utility 
        elif player == 'O':
            return -board.utility
        else:
            return 0
 
    def make_move(self, board, square):
        player = board.to_move
        if player == 'O':
            board = board.update_board({square: player}, to_move='X')
        else:
            board = board.update_board({square: player}, to_move='O')
        win = k_in_row(board, player, square, self.k)
        if win:
            if player == 'X':
                board.utility = 1
            else:
                board.utility = -1
            
        else:
            board.utility = 0
        
        return board
 
 
    def end(self, board):
        if (board.utility != 0) or (len(self.squares)==len(board)):
            return True
        else:
            return False
 
    def draw_board(self, board):
        print(board) 




In [5]:
def random_player(game, state):
    return random.choice(list(game.actions(state)))

In [19]:
def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)[1]

In [7]:
def play_game(game, strategies: dict):
    state = game.initial
    while game.end(state)==False:
        player = state.to_move
        move = strategies[player](game, state)
        state = game.make_move(state, move)
    return state

In [20]:
class ConnectFour(TicTacToe):
    
    def __init__(self): 
        super().__init__(height=6, width=7, k=4)
    def actions(self, board):
        return {(x, y) for (x, y) in self.squares - set(board)
                if y == board.height - 1 or (x, y + 1) in board}

### MCTS Algorithm

In [9]:
class MTS_Node:
    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state,U=U, N=N)
        self.children = {}
        self.actions = None

In [10]:
def best_child(n, C=1.4):
    return (float('inf') if n.N == 0 else
        n.U / n.N + C * math.sqrt(math.log(n.parent.N)/n.N))

In [21]:
def monte_carlo_tree_search(state, game, N=1000):
    def select(node):
        if node.children:
            return select(max(node.children.keys(), key=best_child))  
        else:
            return node 
 
    def expand(node):
        if not node.children and not game.end(node.state):
            node.children = {MTS_Node(state=game.make_move(node.state, action), parent=node): action
                          for action in game.actions(node.state)}
        return select(node)  
 
    def UCT_search(game, state):
        player = game.to_move(state)
        while not game.end(state):
            action = random.choice(list(game.actions(state)))
            state = game.make_move(state, action)
        v = game.utility(state, player)
        return -v
 
    def backup(n, utility):  
        if utility > 0:
            n.U += utility
        n.N += 1
        if n.parent:
            backup(n.parent, -utility)
 
    root = MTS_Node(state=state)
 
    for _ in range(N):
        leaf = select(root)
        child = expand(leaf)
        result = UCT_search(game, child.state)
        backup(child, result)
 
    max_state = max(root.children, key=lambda p: p.N)
 
    return root.children.get(max_state)

In [22]:
def query_player(game, state):
    print("Move writing the input in the format (x, y). To exit the game enter character")
    print("current state:")
    game.draw_board(state)
    print("Legal Move: {}".format(game.actions(state)))
    print("")
    move = None
    if game.actions(state):
        move_string = input('Move: ')
        try:
            move = eval(move_string)
        except NameError:
            move = move_string
    else:
        print('Illegal')
    return move

In [23]:
def monte_carlo_player(game, state):
    return monte_carlo_tree_search(state, game)

In [24]:
play_game(ConnectFour(), dict(X=random_player, O=monte_carlo_player)).utility

1

In [15]:
play_game(ConnectFour(), dict(X=query_player, O=monte_carlo_player)).utility

Move writing the input in the format (x, y). To exit the game enter character
current state:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Legal Move: {(5, 5), (6, 5), (1, 5), (4, 5), (0, 5), (2, 5), (3, 5)}



Move:  (5,5)


Move writing the input in the format (x, y). To exit the game enter character
current state:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X O

Legal Move: {(1, 5), (5, 4), (6, 4), (4, 5), (0, 5), (2, 5), (3, 5)}



Move:  (5,4)


Move writing the input in the format (x, y). To exit the game enter character
current state:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .
. O . . . X O

Legal Move: {(6, 4), (1, 4), (4, 5), (0, 5), (5, 3), (2, 5), (3, 5)}



...

Move writing the input in the format (x, y). To exit the game enter character
current state:
. . . . . . .
. . . . . . .
. . . . . . O
. . . . O X X
. O . . O X X
. O . . X X O

Legal Move: {(6, 1), (4, 2), (0, 5), (2, 5), (1, 3), (3, 5), (5, 2)}



Move:  (6,1)


Move writing the input in the format (x, y). To exit the game enter character
current state:
. . . . . . .
. . . . . . X
. . . . O . O
. . . . O X X
. O . . O X X
. O . . X X O

Legal Move: {(4, 1), (0, 5), (6, 0), (2, 5), (1, 3), (3, 5), (5, 2)}



Move:  (6,0)


-1